In [4]:
# Import packages
import pandas as pd
import numpy as np
import datetime
from collections import defaultdict
import nltk
import random


In [7]:
stemmer = nltk.stem.snowball.SnowballStemmer("english")
data_dir = "/Users/yangsong/Desktop/Projects/gitrepo_songyang0716/Topic_Modeling/reviews_small.txt"
np.random.seed(666)

In [8]:
# read review texts
reviews = []
f = open(data_dir, "r")
for review in f:
    reviews.append(review)
random.shuffle(reviews)

In [9]:
# process text
# tokenize, lower, remove stop words, stem, then only keep alphabets in the string
clean_reviews = []
for review in reviews:
    s = nltk.word_tokenize(review)
    s = [word.lower() for word in s]
    s = [word for word in s if not word in set(
        nltk.corpus.stopwords.words('english'))]
    s = [stemmer.stem(word) for word in s if word.isalpha()]
    clean_reviews.append(" ".join(s))

In [10]:
len(clean_reviews)

147

In [11]:
reviews[0]

"This is the second time I've used APlus for plumbing issues. Juan and Miguel once again resolved our problem with ease and efficiency. I was happy to see them waiting for me once I got home from work at 4:07p for my 4p-6 appointment window.\n"

In [12]:
clean_reviews[0]

'second time use aplus plumb issu juan miguel resolv problem eas effici happi see wait got home work appoint window'

In [54]:
biterms = []
for clean_review in clean_reviews:
    clean_review = clean_review.split()
    review_length = len(clean_review)
    cur_review_biterms = set()
    for i in range(review_length):
        for j in range(i+1, review_length):
            cur_review_biterms.add((clean_review[i], clean_review[j]))
    biterms.extend(list(cur_review_biterms))

In [55]:
len(biterms)

104242

In [56]:
biterms[0]

('juan', 'wait')

In [57]:
N_BITERMS = len(biterms)

In [58]:
N_BITERMS

104242

In [60]:
num_of_topics = 3

In [61]:
biterm_topic = np.random.randint(0, num_of_topics, N_BITERMS)

In [63]:
biterm_topic

array([0, 2, 1, ..., 2, 1, 2])

In [66]:
np.unique(biterm_topic, return_counts=True)[0]

array([0, 1, 2])

In [67]:
np.unique(biterm_topic, return_counts=True)[1]

array([34752, 34830, 34660])

In [70]:
biterm_topic[0] = -1

In [74]:
nz = np.unique(biterm_topic, return_counts=True)[1][1:]

In [75]:
nz

array([34751, 34830, 34660])